#### Project
- 낚시성 기사 탐지
- data -> Fake : 낚시성 기사, Real : 일반 기사

In [9]:
pip install Korpora

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from Korpora import Korpora
import json
import os 
import torch
from collections import Counter
from konlpy.tag import *

In [11]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cuXXX

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cuXXX
Note: you may need to restart the kernel to use updated packages.


In [12]:
# 장치 사용
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {DEVICE}")

Using device: cuda


In [13]:
TRAIN_PATH = "./data/Train/"

# TRAIN_DATA = "./data/Train"
# VAL_DATA = "./data/Validation"

label_list = os.listdir(TRAIN_PATH)
print(label_list)

['Fake', 'Real']


In [14]:
dict = {'text': [], 'label': []}

for label in label_list:
    folder_path = os.path.join(TRAIN_PATH, label)
    file_list = os.listdir(folder_path)
    
    for file in file_list:
        file_path = os.path.join(folder_path, file)
        
        with open(file_path, encoding='utf-8') as f:
            data = json.load(f)
            df = pd.json_normalize(data, record_path=["sourceDataInfo", "sentenceInfo"])
            
            full_article = ' '.join(df['sentenceContent'])
            
            dict['text'].append(full_article)
            dict['label'].append(label)

In [15]:
trainDF = pd.DataFrame(dict)
trainDF['label'] = trainDF['label'].replace({'Fake':0, 'Real':1})
trainDF.head(7)

C:\Users\DELL-PC\AppData\Local\Temp\ipykernel_18096\1435753867.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  trainDF['label'] = trainDF['label'].replace({'Fake':0, 'Real':1})


,text,label
0,손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분...,0
1,Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향(env...,0
2,요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기(nutriti...,0
3,생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가슴...,0
4,자원봉사는 일반적으로 '기업의 사회적 책임'(CSR)의 주요 구성 요소이다. 직원들...,0
5,백제세계유산센터는 지난달 30일 충남과 전북의 8개 기업체와 5개 지방자치단체가 백...,0
6,삼성전자가 내셔널지오그래픽과멸종 위기에 처한 동물을 위한 특별 전시와 기부 캠페인을...,0


In [16]:
trainDF.tail(7)

,text,label
31884,어쩌다 가족 김하율 지음 l 폴앤니나(2021) 다른 이를 나보다 우선 순위에 놓고...,1
31885,미국인 이야기 1: 독립의 여명(1763~1770) 미국인 이야기 2: 전쟁의 서막...,1
31886,☞뉴스레터 공짜 구독하기 https://bit.ly/319DiiE 1486년 이탈리...,1
31887,서울 마포구 홍익대 앞 인디신을 대표하는 축제로 자리 잡은 ‘경록절’이 2년 연속 ...,1
31888,사단법인 ‘봄’ 이사장인 원로건축인 김원(79)씨가 북한 어린이들을 돕기 위해 소장...,1
31889,"북쪽으론 황해도, 동쪽으론 강원도, 남쪽으론 충남과 접해 있고 중앙에는 광역시 2곳...",1
31890,윤이상과 김순남은 1917년생 동갑내기 작곡가다. 굴곡진 현대사에 휩쓸린 두 음악 ...,1


- train, text data 나누기

In [17]:
from sklearn.model_selection import train_test_split

X = trainDF[['text']]
y = trainDF['label']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 데이터 확인
print(f"Train 데이터 개수: {len(X_train)}")
print(f"Test 데이터 개수: {len(X_test)}")
print(f"총 Data 수 : {len(X_train) + len(X_test)}")

Train 데이터 개수: 25512
Test 데이터 개수: 6379
총 Data 수 : 31891


- 구두점, 불용어 제거

In [18]:
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
# 구두점 제거 함수
import re

def remove_punctuation(text):
    if isinstance(text, str) :
        return re.sub(r"[^\w\s]", "", text)
    else :
        return text

In [20]:
# Train 데이터에서 구두점 제거
X_train = X_train.apply(remove_punctuation)

# Test 데이터에서 구두점 제거
X_test = X_test.apply(remove_punctuation)

In [21]:
# 제거된 결과 확인
print("구두점 제거 후 Train 데이터 샘플:")
X_train.head()

구두점 제거 후 Train 데이터 샘플:


,text
8146,촉망받았던 젊은 바이올리니스트 권혁주씨가 급성심정지로 별세했다. 향년 31. 권씨는...
5066,국내 최고 자리에서 밑바닥을 각오해야 하는 미국 진출이 두렵지 않았을까. 배우 이병...
10213,21세기에도 가슴을 뛰게 하는 멋진 소녀들 대부분은 성차별이 심했던 19~20세기를...
3281,는 ‘일과 사람’ 시리즈의 세 번째 권으로 소방관이라는 직업에 대해 자세하게 알려 ...
19046,2분44초 분량의 통화 녹음파일이 세상을 떠들썩하게 했다. 남양유업 영업사원 A씨가...


In [22]:
print("구두점 제거 후 Test 데이터 샘플:")
print(X_test.head())

구두점 제거 후 Test 데이터 샘플:
                                                    text
12846  빅토르 E. 프랑클 지음, 박현용 옮김/책세상·1만2000원 빅토르 프랑클이 쓴 의...
9879   소설&지도 앤드루 더그라프·대니얼 하먼 지음, 한유주 옮김/비채·2만2000원 소설...
2538   ■ 왕년의 명엠시 이덕화 토크쇼 부탁해요(SBS 밤 11시10분) 왕년의 명엠시 이...
9654   “배우와 영화의 관계는 마치 ‘연애’와 같아요. 애타게 찾을 땐 다가오지 않다가 어...
21966  약초의 본고장 제천이 요즘 약초 냄새로 가득하다. 예로부터 일교차와 석회암의 사질토...


- 불용어 제거

In [23]:
# 불용어 리스트 생성
stop_path = "./data/stop_words.txt"

In [24]:
with open(stop_path, "r", encoding="utf-8") as f:
    wordlist = f.readlines()

wordlist

['프\n',
 '뗴\n',
 '잠시\n',
 '채\n',
 '즉시\n',
 '드\n',
 '하도록시키다\n',
 '제\n',
 '하는바\n',
 '쓰\n',
 '으로써\n',
 '연이서\n',
 '삐\n',
 '결론을 낼 수 있다\n',
 '지만\n',
 '조용히\n',
 '보는데서\n',
 'ㅟ\n',
 'ㅘ\n',
 '탸\n',
 '깨\n',
 '똬\n',
 '여보세요\n',
 '잇따라\n',
 'ㅐ\n',
 '삼\n',
 '거바\n',
 '이 때문에\n',
 '소생\n',
 '쉿\n',
 '함께\n',
 '난\n',
 '정도에 이르다\n',
 '언제\n',
 '괴\n',
 '매번\n',
 '하는 김에\n',
 '거기\n',
 '초\n',
 '까\n',
 '바꾸어말하자면\n',
 '솨\n',
 '처\n',
 '꾸\n',
 '까닭으로\n',
 '그럼에도\n',
 '아이참\n',
 '곳\n',
 '더욱더\n',
 '그중에서\n',
 '흥\n',
 '양자\n',
 '차\n',
 'ㅈ\n',
 '라서\n',
 '나\n',
 '비로소\n',
 '쐬\n',
 '요만한걸\n',
 '이 외에\n',
 '허\n',
 '하는 것도\n',
 '겨\n',
 '절대\n',
 '지든지\n',
 '하물며\n',
 '요만한 것\n',
 '좌\n',
 '지말고\n',
 '이리하여\n',
 '좋아\n',
 '이었다\n',
 '하네요\n',
 '따위\n',
 '아하\n',
 '할때\n',
 '그렇지만\n',
 '두\n',
 '어떻게\n',
 '내일\n',
 '퐈\n',
 '료\n',
 '무엇때문에\n',
 '종\n',
 '게\n',
 '버\n',
 '하기에\n',
 '하기 보다는\n',
 '한켠으로는\n',
 '시각\n',
 '펴\n',
 '만은 아니다\n',
 '심지어\n',
 '설\n',
 '도달하다\n',
 '퍽\n',
 'ㅒ\n',
 '겹\n',
 '어기여차\n',
 '에도\n',
 '트\n',
 '타인\n',
 '나왔는데\n',
 '볘\n',
 '퇴\n',
 '세상에\n',
 '

In [25]:
stopwords = []

for word in wordlist :
    stopwords.append(word.replace("\n", ""))
print(len(wordlist), len(stopwords))
print(stopwords[-10:])

1193 1193
['만일', '상대적으로 말하자면', '더구나', '붕붕', '바꾸어말하면', '주룩주룩', '한항목', '예를 들면', '아야', '지금']


In [26]:
# 토큰화
counter = Counter()
tokenizer = Okt()

In [27]:
train_tokens = [tokenizer.morphs(review) for review in X_train.text]
test_tokens = [tokenizer.morphs(review) for review in X_test.text]

In [28]:
len(train_tokens), len(test_tokens)

(25512, 6379)

- 단어사전 생성

In [29]:
def build_vocab(corpus, n_vocab, special_tokens) :
    counter = Counter()
    for tokens in corpus :
        counter.update(tokens)
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab) :
        vocab.append(token)
    return vocab

In [30]:
vocab = build_vocab(corpus = train_tokens, n_vocab = 20000, special_tokens = ["<PAD>", "<UNK>"])
token_to_id = {token:idx for idx, token in enumerate(vocab)}
id_to_token = {idx:token for idx, token in enumerate(vocab)}

print(vocab[:20])
print(len(vocab))

print(token_to_id)
print(id_to_token)

['<PAD>', '<UNK>', '.', '을', '의', '이', ',', '에', '를', '은', '가', '는', '들', '한', '‘', '으로', '에서', '’', '과', '것']
20002
{'<PAD>': 0, '<UNK>': 1, '.': 2, '을': 3, '의': 4, '이': 5, ',': 6, '에': 7, '를': 8, '은': 9, '가': 10, '는': 11, '들': 12, '한': 13, '‘': 14, '으로': 15, '에서': 16, '’': 17, '과': 18, '것': 19, '로': 20, '(': 21, '적': 22, ')': 23, '하는': 24, '”': 25, '“': 26, '이다': 27, '했다': 28, '도': 29, '그': 30, '와': 31, '인': 32, '등': 33, '수': 34, '·': 35, '있다': 36, '다': 37, '고': 38, '한다': 39, '하고': 40, '할': 41, '해': 42, '말': 43, '있는': 44, '된': 45, "'": 46, '사람': 47, '에게': 48, '때': 49, '한국': 50, '나': 51, '책': 52, '까지': 53, '전': 54, '\\"': 55, '에는': 56, '영화': 57, '대': 58, '만': 59, '대한': 60, '때문': 61, '부터': 62, '더': 63, '위': 64, '된다': 65, '못': 66, '제': 67, '성': 68, '일': 69, '우리': 70, '내': 71, '중': 72, '작품': 73, '1': 74, '게': 75, '작가': 76, '함께': 77, '같은': 78, '자신': 79, '이야기': 80, '생각': 81, '뒤': 82, '명': 83, '되는': 84, '위해': 85, '아이': 86, '통해': 87, '이라고': 88, '?': 89, '사회': 90, '2': 91, '처럼': 92, '시작': 93,

- 정수 인코딩 및 패딩

In [31]:
def pad_sequences(sequences, max_length, pad_value) :
    result = list()
    for sequence in sequences :
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

In [32]:
import numpy as np 

unk_id = token_to_id['<UNK>']
train_ids = [[token_to_id.get(token, unk_id)for token in text] for text in train_tokens]
test_ids = [[token_to_id.get(token, unk_id)for token in text] for text in test_tokens]

max_length = 200
pad_id = token_to_id["<PAD>"]
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[18705  3926   535  5322     1   126    10  3404     1    20  7464    28
     2  8930  4660     2 17254    11  1977  2291   881    16  5156     8
  1541 11496    72  6055   466   478   102     2 17254    11     5   181
   885   881   105  2456    16   422    45   144     4 17048    20  1856
   191  1899   881     7 16258     6  5156     8  1541  7493    20   461
   635   421   177     2 17254    11  3033  1360  7560  9734     3  1858
    40     6  2275  5032  8636     1  2780  1952     6  1360   384     1
  2780  1952    16     1    33 18705   582  7909  1483   312     2   146
   161    49  2780     7  4686    42  3208   161     7    50   273  1181
   411  4849   411     7  3883    42     1    99     8 14858    28     2
  4445   161     7  1360    20  2342     3  2521  7870   585   148   411
    16  2782    28     2  1053    82  7560   757   384  1952    16     1
  2780 14405    20    95   572     3  5279     2  7560   134  4206     6
 16835  2115  4597     6  9553  4850   790  1707   

- Data Loader

In [33]:
import torch
from torch.utils.data import TensorDataset, DataLoader

train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(trainDF.loc[X_train.index, 'label'].values, dtype=torch.float32)
test_labels = torch.tensor(trainDF.loc[X_test.index, 'label'].values, dtype=torch.float32)

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [34]:
torch.cuda.is_available()

True

In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")


Using device: cuda


- 학습모델

In [39]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

class SentenceClassifier(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, n_layers, dropout=0.5, bidirectional=True, model_type="lstm", pretrained_embedding=None):
        super().__init__()
        if pretrained_embedding is not None:
            self.embedding = nn.Embedding.from_pretrained(
                torch.tensor(pretrained_embedding, dtype=torch.float32),
                padding_idx=0
            )
        else:
            self.embedding = nn.Embedding(
                num_embeddings=n_vocab,
                embedding_dim=embedding_dim,
                padding_idx=0
            )
        
        # LSTM 모델 사용
        self.model = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )

        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

# 모델 초기화
n_vocab = len(token_to_id)  # 사전 크기
hidden_dim = 674  # LSTM 은닉층 크기
embedding_dim = 128  # Embedding 차원
n_layers = 2  # LSTM 레이어 수

device = "cuda" if torch.cuda.is_available() else "cpu"

classifier = SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

# 모델 저장을 위한 경로
SAVE_FILE = "best_model_state_dict.pth"
SAVE_MODEL = "best_model.pth"

best_val_loss = float('inf')

# 학습 설정
epochs = 100
interval = 500


# 학습 함수 정의
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = []
    corrects = []

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        # 예측값 계산 (sigmoid로 확률화하고 0.5를 기준으로 이진 분류)
        predictions = torch.sigmoid(logits) > 0.5
        corrects.extend(torch.eq(predictions, labels).cpu().tolist())  # 정답과 예측 비교

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            train_loss = np.mean(losses)  # 현재까지의 평균 손실 계산
            train_accuracy = np.mean(corrects)  # 현재까지의 평균 정확도 계산
            print(f"Train Step {step}: Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")


# 학습 및 테스트 반복
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train(classifier, train_loader, criterion, optimizer, device, interval)

    

# 테스트 함수 정의
def test(model, datasets, criterion, device):
    global best_val_loss  # 전역 변수로 설정하여 저장 기준으로 사용
    model.eval()
    losses = []
    corrects = []

    with torch.no_grad():
        for step, (input_ids, labels) in enumerate(datasets):
            input_ids = input_ids.to(device)
            labels = labels.to(device).unsqueeze(1)

            logits = model(input_ids)
            loss = criterion(logits, labels)
            losses.append(loss.item())

            yhat = torch.sigmoid(logits) > 0.5
            corrects.extend(torch.eq(yhat, labels).cpu().tolist())

    val_loss = np.mean(losses)
    val_accuracy = np.mean(corrects)

    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # 가장 좋은 성능일 때 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), SAVE_FILE)  # state_dict 저장
        torch.save(model, SAVE_MODEL)  # 전체 모델 저장
        print(f"Best model saved with loss: {val_loss:.4f}")

# 학습 및 테스트 반복
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Epoch 1/100
Train Step 0: Loss: 0.6892, Accuracy: 0.5625
Train Step 500: Loss: 0.7107, Accuracy: 0.5195
Train Step 1000: Loss: 0.7039, Accuracy: 0.5228
Train Step 1500: Loss: 0.7004, Accuracy: 0.5295
Epoch 2/100
Train Step 0: Loss: 0.6468, Accuracy: 0.6875
Train Step 500: Loss: 0.6992, Accuracy: 0.5467
Train Step 1000: Loss: 0.6967, Accuracy: 0.5387
Train Step 1500: Loss: 0.6957, Accuracy: 0.5368
Epoch 3/100
Train Step 0: Loss: 0.7882, Accuracy: 0.5000
Train Step 500: Loss: 0.6930, Accuracy: 0.5488
Train Step 1000: Loss: 0.6882, Accuracy: 0.5496
Train Step 1500: Loss: 0.6871, Accuracy: 0.5511
Epoch 4/100
Train Step 0: Loss: 0.6382, Accuracy: 0.5625
Train Step 500: Loss: 0.6684, Accuracy: 0.5770
Train Step 1000: Loss: 0.6707, Accuracy: 0.5722
Train Step 1500: Loss: 0.6721, Accuracy: 0.5728
Epoch 5/100
Train Step 0: Loss: 0.6572, Accuracy: 0.5625
Train Step 500: Loss: 0.6466, Accuracy: 0.6005
Train Step 1000: Loss: 0.6521, Accuracy: 0.5973
Train Step 1500: Loss: 0.6537, Accuracy: 0.5950


KeyboardInterrupt: 

- 시각화

In [40]:
import matplotlib.pyplot as plt

# 손실(Loss) 시각화
def plot_loss(LOSS_HISTORY):
    plt.figure(figsize=(10, 5))
    
    # 훈련 손실
    plt.plot(LOSS_HISTORY['train'], label='Train Loss', color='blue', marker='o')
    
    # 검증 손실
    plt.plot(LOSS_HISTORY['val'], label='Validation Loss', color='orange', marker='o')
    
    plt.title("Train vs Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

# F1 Score 시각화
def plot_f1_score(SCORE_HISTORY):
    plt.figure(figsize=(10, 5))
    
    # 훈련 F1 Score
    plt.plot(SCORE_HISTORY['train'], label='Train F1 Score', color='blue', marker='o')
    
    # 검증 F1 Score
    plt.plot(SCORE_HISTORY['val'], label='Validation F1 Score', color='orange', marker='o')
    
    plt.title("Train vs Validation F1 Score")
    plt.xlabel("Epochs")
    plt.ylabel("F1 Score")
    plt.legend()
    plt.grid(True)
    plt.show()

# 손실 및 F1 스코어 시각화
plot_loss(LOSS_HISTORY)
plot_f1_score(SCORE_HISTORY)

NameError: name 'LOSS_HISTORY' is not defined